# A IPython Notebook to Analyze LA Active Business Data

As part of the effort to encourage data-journalism and informed activism, the city of Los Angeles partnered with Socrata to share public data on registered local businesses.

https://data.lacity.org/A-Prosperous-City/Listing-of-Active-Businesses/

This notebook is an attempt to show how the toolkit from the Python stack can be used for a real world data analysis that can allow an LA neighborhood resident to learn more about their community. In this example, we will be building a map of businesses and associated registration and purpose information into a browsable map.

To view a fully interactive version of this Jupyter Notebook, you can set up a notebook server locally. For more information, see Jupyter's official documentation.

In [1]:
# Make sure to install these packages before running:

# pip install sodapy
# pip install folium

import folium

from sodapy import Socrata

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.lacity.org", None)

In [3]:
# First 5000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ngkp-kqkn", limit=5000, city='VENICE')

client.close()

In [4]:
# Examine first record returned in the result set

results[0]

{'business_name': 'CHARCOAL VENICE PARTNERS LLC',
 'city': 'VENICE',
 'council_district': '11',
 'dba_name': 'CHARCOAL VENICE',
 'location_1': {'coordinates': [-118.4627, 33.9815], 'type': 'Point'},
 'location_account': '0002798053-0001-3',
 'location_description': '425 WASHINGTON 90292-5213',
 'location_start_date': '2015-01-18T00:00:00.000',
 'mailing_address': '425 WASHINGTON BLVD',
 'mailing_city': 'VENICE',
 'mailing_zip_code': '90292-5213',
 'naics': '722110',
 'primary_naics_description': 'Full-service restaurants',
 'street_address': '425 WASHINGTON BLVD',
 'zip_code': '90292-5213'}

In [5]:
def flatten(results):
    '''Flatten coordinates in nested dictionary'''
    new_dict = results.copy()
    try:
        coordinates = new_dict['location_1']['coordinates']
    except AttributeError:
        return
    new_dict.pop('location_1', None)
    new_dict['coordinates'] = coordinates
    return new_dict

In [6]:
def reverse_coordinates(results):
    '''Reverse coordinates in flatted dictionary for Folium's use'''
    new_dict = results.copy()
    try:
        coordinates = results['coordinates']
        reversed_coordinates = coordinates[::-1]
        new_dict['coordinates'] = reversed_coordinates
    except AttributeError:
        return
    return new_dict

In [38]:
# Create folium Map object centered around Venice, CA
m = folium.Map(
    location=[33.9889, -118.4717],
    zoom_start=17)

In [39]:
m

In [40]:
# Iterate through list of businesses in returned data set
# and place a marker on the map with select metadata
for business in results:
    try:
        data = reverse_coordinates(flatten(business))
        info_html_string = '<b> {business} </b> d.b.a. {dba} - {description}'.format(business=data['business_name']
                    ,description=data['primary_naics_description']
                    ,dba=data['dba_name'])
        info_html_string = info_html_string.replace("'", r"\'")
        folium.Marker(data['coordinates'],popup=info_html_string).add_to(m)
    except KeyError:
        pass

In [41]:
m

In [42]:
# Save the output of folium.Map object to an HTML file

m.save('venice_map.html')